In [1]:
import xarray as xr
from metpy.constants import g, omega as om, Re as a
from metpy.units import units
import netCDF4
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [12]:
# Upload Observations seasonally average anomalies

dir = '/chinook/dallmann/Ob_data/Ob_data/'
fn_ob_anom = 'precip.V1.0.mon.anom.nc'


data_pr = xr.open_dataset(dir+fn_ob_anom)

# Parse using MetPy, so we can use MetPy operations with it
#data_had_pr_his = data_had_pr_his.metpy.parse_cf()

# output directory and file
dro = '/chinook/dallmann/Ob_data/ELI_Bins/'
fn_output = 'pr_nino_strong_obs_anomDJFmean.nc'

In [3]:
# Upload the array for Nino_strong
dir = '/chinook/dallmann/ENSO_bins/'
fn_ENSO = 'Obs_Nino_strong_DJFmean.nc'

data_ENSO = xr.open_dataset(dir+fn_ENSO)
data_ENSO

<xarray.Dataset>
Dimensions:  (year: 4)
Coordinates:
  * year     (year) int64 1878 1941 1983 1998
Data variables:
    nino     (year) float32 ...

In [11]:
# Convert time to year so it can be found in an Integer64
# I am unsure if this is right since it already was seasonally averaged (unsure what to do about the .mean())
time = data_pr.groupby('time.year').mean()
# shift the temp data by one year because cdo reconizes the year assoicated with december. 
time = time.shift(year=1)

time

<xarray.Dataset>
Dimensions:  (lat: 120, lon: 300, year: 45)
Coordinates:
  * lon      (lon) float32 230.1 230.4 230.6 230.9 ... 304.1 304.4 304.6 304.9
  * lat      (lat) float32 20.12 20.38 20.62 20.88 ... 49.12 49.38 49.62 49.88
  * year     (year) float64 1.948e+03 1.949e+03 1.95e+03 ... 1.991e+03 1.992e+03
Data variables:
    precip   (year, lat, lon) float32 nan nan nan nan nan ... nan nan nan nan

In [5]:
pr_nino_strong = time.where(time['year'] == data_ENSO['year'], drop = True)
pr_nino_strong

<xarray.Dataset>
Dimensions:  (lat: 120, lon: 300, year: 1)
Coordinates:
  * year     (year) float64 1.983e+03
  * lon      (lon) float32 230.1 230.4 230.6 230.9 ... 304.1 304.4 304.6 304.9
  * lat      (lat) float32 20.12 20.38 20.62 20.88 ... 49.12 49.38 49.62 49.88
Data variables:
    precip   (year, lat, lon) float32 nan nan nan nan nan ... nan nan nan nan

In [6]:
# Create a new netcdf file for preciptiation relative to this respective year
pr_nino_strong.to_netcdf(path=dro+fn_output)